# 🤖 Workshop: สร้าง AI ตอบคำถามจากเอกสาร ด้วย RAG Framework
### ดร. CJ - Walailak University
เวิร์กชอปนี้ออกแบบสำหรับนักเรียนมัธยมปลาย เพื่อให้เข้าใจแนวคิด RAG (Retrieval-Augmented Generation) และลองสร้างระบบ AI ตอบคำถามจากเอกสารได้ด้วยตัวเอง

In [1]:
!pip install sentence-transformers faiss-cpu openai -q
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import numpy as np

print('✅ เครื่องมือพร้อมใช้งานแล้ว!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.7 MB/s eta 0:00:00
✅ เครื่องมือพร้อมใช้งานแล้ว!


## 🧩 ส่วนที่ 1: สร้างเอกสารตัวอย่าง

In [2]:
text = '''
สมองของมนุษย์ประกอบด้วยเซลล์ประสาทนับพันล้านเซลล์
ซึ่งส่งสัญญาณไฟฟ้าเพื่อให้เราคิด วิเคราะห์ และจดจำสิ่งต่าง ๆ ได้
ระบบประสาททำงานร่วมกับประสาทสัมผัส เช่น การมองเห็นและการได้ยิน
ทำให้มนุษย์สามารถตอบสนองต่อสิ่งรอบตัวได้อย่างรวดเร็ว
'''
print('ตัวอย่างเอกสาร:')
print(text)

ตัวอย่างเอกสาร:

สมองของมนุษย์ประกอบด้วยเซลล์ประสาทนับพันล้านเซลล์
ซึ่งส่งสัญญาณไฟฟ้าเพื่อให้เราคิด วิเคราะห์ และจดจำสิ่งต่าง ๆ ได้
ระบบประสาททำงานร่วมกับประสาทสัมผัส เช่น การมองเห็นและการได้ยิน
ทำให้มนุษย์สามารถตอบสนองต่อสิ่งรอบตัวได้อย่างรวดเร็ว



## ⚙️ ส่วนที่ 2: แบ่งเอกสารเป็นชิ้นเล็ก ๆ (Chunking)

In [4]:
chunk_size = 80
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
print(f'จำนวนชิ้นข้อมูล (chunks): {len(chunks)}')
for i, c in enumerate(chunks):
    print(f'Chunk {i+1}: {c}')

จำนวนชิ้นข้อมูล (chunks): 3
Chunk 1: 
สมองของมนุษย์ประกอบด้วยเซลล์ประสาทนับพันล้านเซลล์
ซึ่งส่งสัญญาณไฟฟ้าเพื่อให้เรา
Chunk 2: คิด วิเคราะห์ และจดจำสิ่งต่าง ๆ ได้
ระบบประสาททำงานร่วมกับประสาทสัมผัส เช่น การม
Chunk 3: องเห็นและการได้ยิน
ทำให้มนุษย์สามารถตอบสนองต่อสิ่งรอบตัวได้อย่างรวดเร็ว



## 🧠 ส่วนที่ 3: ใช้ LLM สร้างสรุป (จำลอง Context Generation)

In [5]:
def fake_summary(chunk):
    return f'สรุป: {chunk[:30]}...'

summaries = [fake_summary(c) for c in chunks]
for s in summaries:
    print(s)

สรุป: 
สมองของมนุษย์ประกอบด้วยเซลล์ป...
สรุป: คิด วิเคราะห์ และจดจำสิ่งต่าง ...
สรุป: องเห็นและการได้ยิน
ทำให้มนุษย์...


## 🔢 ส่วนที่ 4: แปลงข้อความเป็นเวกเตอร์ (Embedding)

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(summaries)
print(f'Embedding ตัวอย่าง: {embeddings[0][:10]}')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding ตัวอย่าง: [-0.0129114  -0.00480566  0.05662291 -0.01247256  0.07354049  0.05213504
  0.1361868  -0.02389931  0.07967992 -0.08262175]


## 🔍 ส่วนที่ 5: สร้างระบบค้นหา (Retriever)

In [7]:
query = 'มนุษย์คิดได้อย่างไร'
query_vec = model.encode([query])
scores = cosine_similarity(query_vec, embeddings)
for i, score in enumerate(scores[0]):
    print(f'Chunk {i+1} | คะแนนความใกล้เคียง: {score:.3f}')

best_idx = np.argmax(scores)
print('🔍 ข้อมูลที่ใกล้เคียงที่สุดคือ:')
print(chunks[best_idx])

Chunk 1 | คะแนนความใกล้เคียง: 0.615
Chunk 2 | คะแนนความใกล้เคียง: 0.535
Chunk 3 | คะแนนความใกล้เคียง: 0.566
🔍 ข้อมูลที่ใกล้เคียงที่สุดคือ:

สมองของมนุษย์ประกอบด้วยเซลล์ประสาทนับพันล้านเซลล์
ซึ่งส่งสัญญาณไฟฟ้าเพื่อให้เรา


## 💬 ส่วนที่ 6: ให้ AI สร้างคำตอบ (จำลอง)

In [10]:
def fake_answer(context, question):
    return f'จากข้อมูล: {context[:50]}... คำตอบ: มนุษย์สามารถคิดได้เพราะสมองมีเซลล์ประสาทจำนวนมากที่เชื่อมโยงกัน'

context = chunks[best_idx]
answer = fake_answer(context, query)
print(' 🤖 คำตอบของ AI: ', answer)

 🤖 คำตอบของ AI:  จากข้อมูล: 
สมองของมนุษย์ประกอบด้วยเซลล์ประสาทนับพันล้านเซลล์... คำตอบ: มนุษย์สามารถคิดได้เพราะสมองมีเซลล์ประสาทจำนวนมากที่เชื่อมโยงกัน


## 🎮 ส่วนที่ 7: ให้นักเรียนลองทำเอง!

In [12]:
user_q = input('พิมพ์คำถามของคุณ: ')
user_vec = model.encode([user_q])
scores = cosine_similarity(user_vec, embeddings)
best_idx = np.argmax(scores)

print('🔍 ข้อมูลที่เกี่ยวข้อง:')
print(chunks[best_idx])

print('🤖 คำตอบจำลองของ AI:')
print(fake_answer(chunks[best_idx], user_q))

พิมพ์คำถามของคุณ: สมองคือ
🔍 ข้อมูลที่เกี่ยวข้อง:

สมองของมนุษย์ประกอบด้วยเซลล์ประสาทนับพันล้านเซลล์
ซึ่งส่งสัญญาณไฟฟ้าเพื่อให้เรา
🤖 คำตอบจำลองของ AI:
จากข้อมูล: 
สมองของมนุษย์ประกอบด้วยเซลล์ประสาทนับพันล้านเซลล์... คำตอบ: มนุษย์สามารถคิดได้เพราะสมองมีเซลล์ประสาทจำนวนมากที่เชื่อมโยงกัน
